In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
import random
import seaborn as sns
import datetime
import pandas as pd
#!pip install sklearn
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
import requests
import json
import time
import re

In [211]:
def get_realtime_info(stock_code):
    URL_link = "https://www.citifirst.com.hk/en/data/json/json_realtimedata/code/"+stock_code
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    soup = BeautifulSoup(requests.get(URL_link, headers=headers).content, 'html.parser')

    start_index = str(soup).find("{")
    end_index = str(soup).find("}")
    st = str(soup)[start_index:end_index+1]
    json_str = ""
    for ln in st.split(","):
        if "<" not in ln:
            json_str = json_str + ln + ","
    dic = json.loads(json_str[:-1])

    stock_info = {}
    stock_info['Open'] = dic['open']
    stock_info['High'] = dic['high']
    stock_info['Low'] = dic['low']
    stock_info['Close'] = dic['last']
    stock_info['previous_Close'] = dic['lastc']
    stock_info['turnover'] = dic['turnover']
    stock_info['date_time'] = dic['stimeNoformat']
    return stock_info

# add week number for the dataframe with date as index
def calculate_week_number(df_data):
    start_weekday = df_data.index[0].weekday() # offset
    start_date = df_data.index[0]
    week_nums = []
    i = 0
    while i<len(df_data.index):
        cur_date = df_data.index[i]
        cur_week_num = ( int((cur_date- start_date).days) + start_weekday ) // 7
        week_nums.append( cur_week_num )
        i += 1
    df_data['week_num'] = week_nums
    return df_data

# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2023-12-31", real_time=True):
    df_data = None
    df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 

    if real_time:
        stock_info = get_realtime_info(ticker_name)
        open_price, high_price, low_price, current_price = stock_info['Open'], stock_info['High'], stock_info['Low'], stock_info['Close']
        try:
            df_data.at[df_data.index[-1], "Open"] = float(open_price)
            df_data.at[df_data.index[-1], "High"] = float(high_price)
            df_data.at[df_data.index[-1], "Low"] = float(low_price)
            df_data.at[df_data.index[-1], "Close"] = float(current_price)
        except:
            print(stock_info)
            df_data.at[df_data.index[-1], "Open"] = float(current_price)
            df_data.at[df_data.index[-1], "High"] = float(current_price)
            df_data.at[df_data.index[-1], "Low"] = float(current_price)
            df_data.at[df_data.index[-1], "Close"] = float(current_price)
    elif ".HK" in ticker_name:
        print("data may late for 15 minutes")
    # basic calculations such as: daily return, the log of Volume, Moving Average
    df_data['previous_Close'] = df_data['Close'].shift(1)
    df_data['daily_return'] = (df_data['Close']-df_data['previous_Close'])/df_data['previous_Close']
    df_data['Volume_log'] = np.log2(df_data['Volume'])
    MA1, MA2 = 5, 20
    df_data['MA1'] = df_data['Close'].rolling(MA1).mean()
    df_data['MA2'] = df_data['Close'].rolling(MA2).mean()
    df_data['daily_range'] = (df_data['High']-df_data['Low'])/df_data['previous_Close'] * 100
    df_data['daily_to'] = (df_data['Open']+df_data['High']+df_data['Low']+df_data['Close'])/4*df_data['Volume'] * 100 # turnover
    df_data['daily_to'] = np.log2(df_data['daily_to'])

    # add week number
    df_data['weekday'] = df_data.index.weekday
    df_data = calculate_week_number(df_data) 
    return df_data

def check_valid(sub_df, printing=True):
    for f in ['Open', 'Close', 'High', 'Low', 'Volume_log']:
        for v in list(sub_df[f]):
            """if math.isnan(v):
                if printing:
                    print( sub_df[['Open', 'Close', 'High', 'Low', 'Volume', 'week_num']], f, "NaN" )
                return False"""
            if math.isinf(v):
                if printing:
                    print( sub_df[['Open', 'Close', 'High', 'Low', 'Volume', 'week_num']], f, "INF" )
                return False
    return True


def weekly_info(df_data, week_num):
    tmp_df = df_data[ df_data['week_num']==week_num ].copy()
    if not check_valid(tmp_df.copy(), False):
        """tmp_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        tmp_df.dropna(inplace=True)
        if len(tmp_df)<=1:
            print( len(tmp_df), "week_num", week_num )
            return []"""
        return []
        
    Len = len(tmp_df)
    start_date, end_date = tmp_df.index[0], tmp_df.index[-1]
    open_price, close_price = tmp_df['Open'][0], tmp_df['Close'][-1]

    high_price, high_date = open_price, start_date
    low_price, low_date = open_price, start_date
    i = 0
    while i<Len:
        cur_date = tmp_df.index[i]
        cur_high, cur_low = tmp_df['High'][i], tmp_df['Low'][i]
        if cur_high>=high_price:
            high_price = cur_high
            high_date = cur_date
        if cur_low<=low_price:
            low_price = cur_low
            low_date = cur_date
        i += 1
    
    # rise-fall ratio
    rise_N, fall_N = len( tmp_df[tmp_df['daily_return']>0] ), len( tmp_df[tmp_df['daily_return']<0] )
    # open avg, open std, 
    open_avg, open_std = tmp_df['Open'].mean(), tmp_df['Open'].std()
    #high avg, high std, 
    high_avg, high_std = tmp_df['High'].mean(), tmp_df['High'].std()
    #low avg, low std, 
    low_avg, low_std = tmp_df['High'].mean(), tmp_df['High'].std()
    # close avg, close std, 
    close_avg, close_std = tmp_df['Close'].mean(), tmp_df['Close'].std()
    # volume avg, volume std
    volume_avg, volume_std = tmp_df['Volume_log'].mean(), tmp_df['Volume_log'].std() # Volume, Volume_log
    # MA
    ma_avg1, ma_avg2 = tmp_df['MA1'].mean(), tmp_df['MA2'].mean()
    #ma_avg1, ma_avg2 = tmp_df['MA1'][-1], tmp_df['MA2'][-1]
    ma_std1, ma_std2 = tmp_df['MA1'].std(), tmp_df['MA2'].std()
    #
    weekly_change = (close_price - tmp_df['previous_Close'][0]) / tmp_df['previous_Close'][0] * 100
    # 
    range_avg, range_std = tmp_df['daily_range'].std(), tmp_df['daily_range'].std()
    turnover_avg, turnover_std = tmp_df['daily_to'].std(), tmp_df['daily_to'].std()

    it_info = [
                week_num, 
                start_date, open_price, end_date, close_price, 
                high_date, high_price, low_date, low_price,  
                rise_N, fall_N, 
                open_avg, open_std, 
                high_avg, high_std,
                low_avg, low_std,
                close_avg, close_std, 
                volume_avg, volume_std, 
                ma_avg1, ma_avg2,
                ma_std1, ma_std2,
                weekly_change,
                range_avg, range_std,
                turnover_avg, turnover_std
            ]
    
    return it_info

# week num, start date, end date, open price, close price, high price, high date, low price, low date
def generate_weekly_df(df_data):
    data_col = []

    weeknum_list = list( set(list(df_data['week_num'])) )
    weeknum_list = sorted(weeknum_list)
    invalid_weeks = []
    for i in weeknum_list[1:]:
        wn = i
        it_info = weekly_info(df_data, wn)
        if len(it_info)>0:
            data_col.append( it_info )
        else:
            invalid_weeks.append( wn )
    print( len(invalid_weeks), "invalid_weeks: ", invalid_weeks )

    col_names = [   
                    'week-num', 
                    'start_date', 'open_price', 'end_date', 'close_price',                     
                    'high_date', 'high_price', 'low_date', 'low_price',                     
                    'rise_N', 'fall_N',
                    'open_avg', 'open_std',
                    'high_avg', 'high_std',
                    'low_avg', 'low_std',
                    'close_avg', 'close_std',
                    'volume_avg', 'volume_std',
                    'ma_avg1', 'ma_avg2',
                    'ma_std1', 'ma_std2',
                    'weekly_change',
                    'range_avg', 'range_std',
                    'turnover_avg', 'turnover_std'
                ]
    weekly_df = pd.DataFrame(data_col, columns = col_names)
    # next week features & labels
    weekly_df['nw_open'] = weekly_df['open_price'].shift(-1)
    weekly_df['nw_high'] = weekly_df['high_price'].shift(-1)
    weekly_df['nw_low'] = weekly_df['low_price'].shift(-1)
    weekly_df['nw_close'] = weekly_df['close_price'].shift(-1)
    return weekly_df

def get_datasets(weekly_df, features, label):
    X_data, y_data = [], []
    i = 0
    while i < len(weekly_df): # 
        row_data = weekly_df.iloc[i]
        x_tmp = []
        for col in features:
            v = row_data[col]
            x_tmp.append( v )
        y_tmp = row_data[label]
        #print( x_tmp, y_tmp )
        X_data.append( x_tmp )
        y_data.append( y_tmp )
        #print( row_data['week-num'], i )
        i += 1
    return X_data, y_data

from sklearn.linear_model import LinearRegression
def train_model(train_X, train_y, printing=True):
    model = LinearRegression().fit(train_X, train_y)

    r_sq = model.score(train_X, train_y)
    if printing:
        print(f"coefficient of determination: {r_sq}")
        print(f"intercept: {model.intercept_}\tslope: {model.coef_}")
    return model

def error_analyze(train_y, y_pred):
    df = pd.DataFrame(columns = ['y_real', 'y_pred'])
    df['y_real'] = train_y
    df['y_pred'] = y_pred
    df['dif'] = (df['y_real'] - df['y_pred'])/df['y_real'] * 100
    df['dif'] = df['dif'].abs()
    return df

def basic_info(df, col='dif'):
    print("max:\t", df[col].max())
    print("min:\t", df[col].min())
    print("median:\t", df[col].median())
    print("mean:\t", df[col].mean())
    print("std:\t", df[col].std())
    print("10%:\t", df[col].quantile(0.10))
    print("25%:\t", df[col].quantile(0.25))
    print("50%:\t", df[col].quantile(0.50))
    print("75%:\t", df[col].quantile(0.75))
    print("90%:\t", df[col].quantile(0.90))
    return

In [242]:
stocks_info = [
    #('HSI', '^HSI', 1),
    ('BABA', '9988.HK', 1),
    ('BIDU', '9888.HK', 1),
    ('JD', '9618.HK', 0.5 * 7.8),
    ('MPNGY', '3690.HK', 0.5 * 7.8),
    ('NTES', '9999.HK', 0.2 * 7.8),
    ('TENCENT', '0700.HK', 1*7.8),
    ('CMB', '3968.HK', 1*7.8),
	('LI', '2015.HK', 0.5 * 7.8),
	('XPEV', '9868.HK', 0.5*7.8),	
	('BILI', '9626.HK', 1 * 7.8),
	('TCOM', '9961.HK', 1 * 7.8),
	('YUMC', '9987.HK', 1*7.8),
	('EDU', '9901.HK', 0.1*7.8), 
    ('NIO', '9866.HK', 1 * 7.8),
	('ZTO', '2057.HK', 1*7.8),
    ('BEKE', '2423.HK', 0.5*7.8),
    ('ZH', '2390.HK', 3 * 7.8), 
    ('WB', '9898.HK', 1*7.8),
    ('MNSO', '9896.HK', 0.5*7.8),
    ('ZLAB', '9688.HK', 0.5*7.8),
    ('TME', '1698.HK', 1*7.8),
    ('SMIC', '0981.HK', 1*7.8),
    ('SenseTime', '0020.HK', 1*7.8),
    ('Kuaishou', '1024.HK', 1*7.8),
    ('Xiaomi', '1810.HK', 1*7.8),
]

st, et = "2020-01-01", "2023-08-31"

features1 = [   
                'open_price', 'close_price', 'high_price', 'low_price', 
                'rise_N', 'fall_N', 
                'open_avg', 'open_std',
                'high_avg', 'high_std',
                'low_avg', 'low_std',
                'close_avg', 'close_std',
                'volume_avg', 'volume_std',
                'ma_avg1', 'ma_avg2',
                'ma_std1', 'ma_std2',
                'weekly_change', 
                'range_avg', 'range_std',
                'turnover_avg', 'turnover_std',
                'nw_open'
            ]
features2  = features1[:-1]

labels12 = ["nw_close", "nw_high", "nw_low"] 

In [255]:
### predict this week
predicted_data = []
for it in stocks_info[:]:# 
    reference_stock, target_stock, ratio = it   

    test_df = get_df_data(target_stock, st, et, True)
    weekly_df = generate_weekly_df(test_df)
    label = 'nw_close'
    
    new_name = reference_stock + "_" + target_stock
    each_row = []
    for label in labels12:
        # training data
        training_data = weekly_df[10:-2].copy()
        X_data, y_data = get_datasets(training_data, features1, label)

        # testing / predicting this week
        testing_data1 = weekly_df[-2:-1].copy()
        X_test1, y_test1 = get_datasets(testing_data1, features1, label)

        # testing / predicting next week
        testing_data2 = weekly_df[-1:].copy()
        X_test2, y_test2 = get_datasets(testing_data2, features1, label)

        # fit model
        model = train_model(X_data, y_data) 
        # error analysis
        y_pred = model.predict(X_data) 
        error_df = error_analyze(y_data, y_pred)
        err1, err2 = round(error_df['dif'].median(), 2), round(error_df['dif'].mean(), 2)

        pred_this_week = model.predict(X_test1)
        y_pred1 = round(pred_this_week[0], 2)

        # 
        last_week_close = round(X_test1[0][1], 2)
        each_row = each_row + [ y_pred1, y_test1[0], err2 ]
    predicted_data.append( [new_name, last_week_close]+each_row ) 

[*********************100%***********************]  1 of 1 completed
5 invalid_weeks:  [51, 52, 58, 103, 109]
coefficient of determination: 0.9861343427226609
intercept: -45.658420100456794	slope: [ -0.08903484   0.27643351   0.57877055  -0.86380256  -3.05053396
  -2.60530671   2.13815912  -0.77471966  -0.16978933   0.34059661
  -0.16978933   0.34059661  -1.73327297  -1.89731554   2.39149429
  18.19793083  -0.27056725   0.12998924  -0.09539441  -0.40080073
  -0.15093653  -0.02458812  -0.02458812 -11.33047368 -11.33047368
   1.17472808]
coefficient of determination: 0.9933799890099754
intercept: -16.654605364073007	slope: [-0.03133773  0.30276649  1.0605286  -0.90850801 -2.1619643  -1.64973814
  0.23317873 -0.35661372 -0.19632228 -0.62114092 -0.19632228 -0.62114092
 -0.01242365 -0.42360382  1.13480577  4.77455482 -0.458102    0.11209824
 -0.26867965 -0.34491893 -0.34331581 -0.13272622 -0.13272622 -4.24413674
 -4.24413674  1.10800843]
coefficient of determination: 0.9933629898843971
inte

In [256]:
col_names1 = [
        "stock-name", 'last_week_close', 
        "close_pred", "close_real", "close_err_range%", 
        "high_pred", "high_real", "high_err_range%", 
        "low_pred", "low_real", "low_err_range%",    
            ]

col_names = col_names1
round_dic = {
        'close_pred': 2, 'high_pred': 2, 'low_pred': 2, 
        'high_real':2, 'low_real':2,
        'close_err_range%': 2, 'high_err_range%':2, 'low_err_range%':2, 
        'nw_close':2, 'nw_high':2, 'nw_low':2,
        'weekly_change%':2 
            }
stock_df = pd.DataFrame( predicted_data, columns=col_names )
stock_df['price_change'] = stock_df['close_real'] - stock_df['last_week_close']
stock_df['weekly_change%'] = stock_df['price_change'] / stock_df['last_week_close'] * 100


show_cols = [
        "stock-name", 'last_week_close', 'price_change', 'weekly_change%',
        "close_pred", "close_real", "close_err_range%", 
        "high_pred", "high_real", "high_err_range%", 
        "low_pred", "low_real", "low_err_range%",
        ]


stock_df.round(round_dic)[ show_cols ] 

,stock-name,last_week_close,price_change,weekly_change%,close_pred,close_real,close_err_range%,high_pred,high_real,high_err_range%,low_pred,low_real,low_err_range%
0,BABA_9988.HK,95.30,-6.05,-6.35,90.01,89.25,4.25,94.89,93.35,2.92,87.44,86.20,2.63
1,BIDU_9888.HK,137.80,-9.40,-6.82,132.64,128.40,3.96,141.00,137.40,2.76,127.00,125.40,2.48
2,JD_9618.HK,146.10,-11.50,-7.87,152.42,134.60,4.79,155.65,145.60,3.32,137.74,134.10,3.30
3,MPNGY_3690.HK,137.20,-5.40,-3.94,142.10,131.80,6.17,144.51,135.90,3.92,129.57,129.10,4.00
4,NTES_9999.HK,163.60,-5.60,-3.42,158.63,158.00,4.17,165.98,163.40,2.59,151.29,156.50,2.82
5,TENCENT_0700.HK,335.80,-6.40,-1.91,333.36,329.40,3.79,342.75,335.80,2.44,319.51,320.60,2.53
6,CMB_3968.HK,34.15,-3.35,-9.81,32.47,30.80,3.60,34.26,33.55,2.35,31.38,30.60,2.32
7,LI_2015.HK,165.40,-1.50,-0.91,144.08,163.90,5.99,156.48,166.80,3.73,145.72,150.00,3.22
8,XPEV_9868.HK,66.10,-3.25,-4.92,58.98,62.85,7.91,68.30,67.00,5.33,56.25,60.80,4.73
9,BILI_9626.HK,131.00,-8.00,-6.11,123.82,123.00,8.92,138.24,131.20,5.16,118.69,118.00,5.15


In [260]:
### predict this & next week
predicted_data = []
for it in stocks_info[:]:# 
    reference_stock, target_stock, ratio = it   

    test_df = get_df_data(target_stock, st, et, True)
    weekly_df = generate_weekly_df(test_df)
    label = 'nw_close'
    
    new_name = reference_stock + "_" + target_stock
    each_row = []
    for label in labels12:
        # training data
        training_data = weekly_df[10:-2].copy()
        X_data, y_data = get_datasets(training_data, features2, label)

        # testing / predicting this week
        testing_data1 = weekly_df[-2:-1].copy()
        X_test1, y_test1 = get_datasets(testing_data1, features2, label)

        # testing / predicting next week
        testing_data2 = weekly_df[-1:].copy()
        X_test2, y_test2 = get_datasets(testing_data2, features2, label)

        # fit model
        model = train_model(X_data, y_data) 
        # error analysis
        y_pred = model.predict(X_data) 
        error_df = error_analyze(y_data, y_pred)
        err1, err2 = round(error_df['dif'].median(), 2), round(error_df['dif'].mean(), 2)

        pred_this_week = model.predict(X_test1)
        y_pred1 = round(pred_this_week[0], 2)
        pred_next_week = model.predict(X_test2)
        y_pred2 = round(pred_next_week[0], 2)

        # 
        last_week_close = round(X_test1[0][1], 2)
        each_row = each_row + [ y_pred1, y_test1[0], y_pred2, err2 ]
    predicted_data.append( [new_name, last_week_close]+each_row ) 

[*********************100%***********************]  1 of 1 completed
5 invalid_weeks:  [51, 52, 58, 103, 109]
coefficient of determination: 0.9818091590870357
intercept: -70.87700733358565	slope: [-0.34188682  1.28193198  0.64028237 -0.44156383 -4.22196873 -3.24418336
  1.05374265 -0.10073856 -0.49915837 -0.5721539  -0.49915837 -0.5721539
 -0.38840098 -0.35613651  3.47086267 -9.91477314  0.04814864  0.16804006
  0.21061212 -0.34903968 -0.11226847 -0.07402202 -0.07402202  2.69135612
  2.69135612]
coefficient of determination: 0.989730350277125
intercept: -40.4408822235107	slope: [-2.69828771e-01  1.25115685e+00  1.11854681e+00 -5.10250667e-01
 -3.26686648e+00 -2.25232926e+00 -7.89647420e-01  2.79088075e-01
 -5.06984534e-01 -1.48205101e+00 -5.06984534e-01 -1.48205101e+00
  1.25606523e+00  1.03004269e+00  2.15287054e+00 -2.17414650e+01
 -1.57487852e-01  1.47987942e-01  1.99469904e-02 -2.96097694e-01
 -3.06843938e-01 -1.79352478e-01 -1.79352478e-01  8.98131172e+00
  8.98131172e+00]
coeffic

In [261]:
col_names2 = [
            "stock-name", 'last_week_close', 
            "close_pred", "close_real", "nw_close", "close_err_range%", 
            "high_pred", "high_real", "nw_high", "high_err_range%", 
            "low_pred", "low_real", "nw_low", "low_err_range%",            
            ]

col_names = col_names2
round_dic = {
            'close_pred': 2, 'high_pred': 2, 'low_pred': 2, 
            'high_real':2, 'low_real':2,
            'close_err_range%': 2, 'high_err_range%':2, 'low_err_range%':2, 
            'nw_close':2, 'nw_high':2, 'nw_low':2,
            'weekly_change%':2 
            }
stock_df = pd.DataFrame( predicted_data, columns=col_names )
stock_df['price_change'] = stock_df['close_real'] - stock_df['last_week_close']
stock_df['weekly_change%'] = stock_df['price_change'] / stock_df['last_week_close'] * 100


show_cols = [
        "stock-name", 'last_week_close', 'price_change', 'weekly_change%',
        "close_pred", "close_real", "close_err_range%", 
        "high_pred", "high_real", "high_err_range%", 
        "low_pred", "low_real", "low_err_range%",
        "nw_close", "nw_high", "nw_low"
        ]


stock_df.round(round_dic)[ show_cols ] 

,stock-name,last_week_close,price_change,weekly_change%,close_pred,close_real,close_err_range%,high_pred,high_real,high_err_range%,low_pred,low_real,low_err_range%,nw_close,nw_high,nw_low
0,BABA_9988.HK,95.30,-8.30,-8.71,90.80,87.00,4.98,95.64,93.35,3.61,88.17,86.20,3.48,86.63,93.82,82.28
1,BIDU_9888.HK,137.80,-11.70,-8.49,136.97,126.10,5.19,144.55,137.40,3.74,130.62,125.40,3.73,127.76,134.74,120.64
2,JD_9618.HK,146.10,-14.90,-10.20,154.79,131.20,5.25,157.99,145.60,3.79,140.38,131.20,4.00,135.34,145.75,125.35
3,MPNGY_3690.HK,137.20,-7.30,-5.32,145.75,129.90,6.31,148.49,135.90,4.19,134.71,129.10,4.65,131.27,137.07,122.65
4,NTES_9999.HK,163.60,-7.10,-4.34,160.88,156.50,4.39,168.48,163.40,2.98,153.75,156.00,3.11,156.94,162.16,148.16
5,TENCENT_0700.HK,335.80,-10.80,-3.22,339.58,325.00,3.93,349.20,335.80,2.67,327.92,320.60,2.81,334.50,342.66,316.84
6,CMB_3968.HK,34.15,-3.20,-9.37,33.18,30.95,4.12,34.89,33.55,2.80,32.10,30.60,3.03,30.01,31.44,28.46
7,LI_2015.HK,165.40,-10.00,-6.05,156.79,155.40,6.81,168.81,166.80,4.79,156.45,150.00,4.34,151.30,159.36,151.76
8,XPEV_9868.HK,66.10,-5.10,-7.72,64.13,61.00,8.59,73.49,67.00,6.51,60.46,60.65,5.62,61.94,65.35,57.09
9,BILI_9626.HK,131.00,-14.10,-10.76,130.17,116.90,10.43,144.42,131.20,6.68,123.91,116.80,6.59,109.98,120.78,100.37
